In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [3]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [4]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [5]:
def my_loss(real, X_pred):
    y_pred = dnn_model(X_pred)
    return tf.math.subtract(y_pred, 42)

In [6]:
def return_latest():
    df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    return X[-1]

In [7]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(256, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(128, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='tanh'))

generator.compile(loss=my_loss, optimizer='adam')

In [8]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    if i % 10 == 0:
        print(f"epoch: {i}, loss: {loss}")

epoch: 0, loss: [[-1.4577484]]
epoch: 10, loss: [[-5.499859]]
epoch: 20, loss: [[-9.957203]]
epoch: 30, loss: [[-15.119354]]
epoch: 40, loss: [[-16.674128]]
epoch: 50, loss: [[-17.354593]]
epoch: 60, loss: [[-17.501606]]
epoch: 70, loss: [[-17.433739]]
epoch: 80, loss: [[-17.45922]]
epoch: 90, loss: [[-17.598505]]
epoch: 100, loss: [[-17.519272]]
epoch: 110, loss: [[-17.500946]]
epoch: 120, loss: [[-17.600372]]
epoch: 130, loss: [[-17.625214]]
epoch: 140, loss: [[-17.718977]]
epoch: 150, loss: [[-17.665142]]
epoch: 160, loss: [[-17.509075]]
epoch: 170, loss: [[-17.801888]]
epoch: 180, loss: [[-17.943739]]
epoch: 190, loss: [[-17.919998]]
epoch: 200, loss: [[-17.894035]]
epoch: 210, loss: [[-17.949715]]
epoch: 220, loss: [[-18.060005]]
epoch: 230, loss: [[-17.987228]]
epoch: 240, loss: [[-18.058344]]
epoch: 250, loss: [[-17.712925]]
epoch: 260, loss: [[-18.086573]]
epoch: 270, loss: [[-18.052265]]
epoch: 280, loss: [[-17.982414]]
epoch: 290, loss: [[-18.125065]]
epoch: 300, loss: [[-17.

In [9]:
result = np.array(result).reshape(1000, 21)
pop_result = np.array(pop_result).reshape(1000, 1)
X_result = scaler.inverse_transform(result)

pd_result = pd.DataFrame(X_result)
pd_result.to_excel('basic.xlsx')

pd_pop_result = pd.DataFrame(pop_result)
pd_pop_result.to_excel('basic_pop.xlsx')